In [14]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from data import ModelNet40
from model import PointNet, DGCNN_cls
import numpy as np
from torch.utils.data import DataLoader
from util import cal_loss, IOStream, vis_point_cloud, vis_point_cloud_side_by_side
import sklearn.metrics as metrics
import pickle
import open3d as o3d

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class args:
    def __init__(self):
        self.batch_size = 32
        self.num_points = 1024
        self.no_cuda = not torch.cuda.is_available()
        self.cuda = torch.cuda.is_available()
        self.seed = 1
        self.exp_name = "cls_1024_eval"
        self.model = "dgcnn"
        self.dataset = "modelnet40"
        self.test_batch_size = 16
        self.epochs = 3 #original 250
        self.lr = 0.001
        self.momentum = 0.9
        self.scheduler = "cos"
        self.dropout = 0.5
        self.k = 20
        self.model_path = "pretrained/model.cls.1024.t7"
        self.emb_dims = 1024
args = args()

if not os.path.exists("outputs"):
    os.makedirs("outputs")
if not os.path.exists("outputs/" + args.exp_name):
    os.makedirs("outputs/" + args.exp_name)
if not os.path.exists("outputs/" + args.exp_name + "/" + "models"):
    os.makedirs("outputs/" + args.exp_name + "/" + "models")
os.system(
    "cp main_cls.py outputs" + "/" + args.exp_name + "/" + "main_cls.py.backup"
)
os.system("cp model.py outputs" + "/" + args.exp_name + "/" + "model.py.backup")
os.system("cp util.py outputs" + "/" + args.exp_name + "/" + "util.py.backup")
os.system("cp data.py outputs" + "/" + args.exp_name + "/" + "data.py.backup")
print(device)

cuda


In [3]:
#sample test
test_loader = DataLoader(
        ModelNet40(partition="test", num_points=args.num_points),
        batch_size=args.test_batch_size,
        shuffle=True,
        drop_last=False,
    )

# Try to load models
if args.model == "pointnet":
    model = PointNet(args).to(device)
elif args.model == "dgcnn":
    model = DGCNN_cls(args).to(device)
else:
    raise Exception("Not implemented")

model = nn.DataParallel(model)
model.load_state_dict(torch.load(args.model_path, map_location=torch.device("cpu")))
model = model.to(device).eval()
test_acc = 0.0
count = 0.0
test_true = []
test_pred = []
for data, label in test_loader:
    count += 1
    print(count / len(test_loader))
    data, label = data.to(device), label.to(device).squeeze()
    data = data.permute(0, 2, 1)
    batch_size = data.size()[0]
    logits = model(data)
    preds = logits.max(dim=1)[1]
    test_true.append(label.cpu().numpy())
    test_pred.append(preds.detach().cpu().numpy())
test_true = np.concatenate(test_true)
test_pred = np.concatenate(test_pred)
test_acc = metrics.accuracy_score(test_true, test_pred)
avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
outstr = "Test :: test acc: %.6f, test avg acc: %.6f" % (
    test_acc,
    avg_per_class_acc,
)
print(outstr)

0.0064516129032258064
0.012903225806451613
0.01935483870967742
0.025806451612903226
0.03225806451612903
0.03870967741935484
0.04516129032258064
0.05161290322580645
0.05806451612903226
0.06451612903225806
0.07096774193548387
0.07741935483870968
0.08387096774193549
0.09032258064516129
0.0967741935483871
0.1032258064516129
0.10967741935483871
0.11612903225806452
0.12258064516129032
0.12903225806451613
0.13548387096774195
0.14193548387096774
0.14838709677419354
0.15483870967741936
0.16129032258064516
0.16774193548387098
0.17419354838709677
0.18064516129032257
0.1870967741935484
0.1935483870967742
0.2
0.2064516129032258
0.2129032258064516
0.21935483870967742
0.22580645161290322
0.23225806451612904
0.23870967741935484
0.24516129032258063
0.25161290322580643
0.25806451612903225
0.2645161290322581
0.2709677419354839
0.27741935483870966
0.2838709677419355
0.2903225806451613
0.2967741935483871
0.3032258064516129
0.3096774193548387
0.3161290322580645
0.3225806451612903
0.32903225806451614
0.33548

In [25]:
#train
train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=True,
    )
test_loader = DataLoader(
    ModelNet40(partition="test", num_points=args.num_points),
    num_workers=8,
    batch_size=args.test_batch_size,
    shuffle=True,
    drop_last=False,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Try to load models
if args.model == "pointnet":
    model = PointNet(args).to(device)
elif args.model == "dgcnn":
    model = DGCNN_cls(args).to(device)
else:
    raise Exception("Not implemented")

# print(str(model))

model = nn.DataParallel(model)
print("Let's use", torch.cuda.device_count(), "GPUs!")

opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)

if args.scheduler == "cos":
    scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-3)
elif args.scheduler == "step":
    scheduler = StepLR(opt, step_size=20, gamma=0.7)

criterion = cal_loss
best_test_acc = 0

EPSILON = 0.2
epsilons = [0, 0.02, 0.05, 0.1, 0.2]
accuracies = {}

for e in epsilons:
    model = DGCNN_cls(args).to(device)
    model = nn.DataParallel(model)
    opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)
    if args.scheduler == "cos":
        scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-3)
    elif args.scheduler == "step":
        scheduler = StepLR(opt, step_size=20, gamma=0.7)

    for epoch in range(args.epochs):
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            data = data.permute(0, 2, 1)
            data.requires_grad = True
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            train_loss += loss.item() * batch_size
            vis_data = data.permute(0, 2, 1).cpu().detach().numpy()

            #perform fgsm
            data_grad = data.grad.data
            perturbed = data + e * data_grad.sign()
            vis_data_perturbed = perturbed.permute(0, 2, 1).cpu().detach().numpy()

            # vis_point_cloud_side_by_side(vis_data[0], vis_data_perturbed[0])
            
            #training model with fgsm data
            opt.zero_grad()
            perturbed_logits = model(perturbed)
            loss = criterion(perturbed_logits, label)
            loss.backward()
            opt.step()
            train_loss += loss.item() * batch_size

            #calculate accuracy
            preds = logits.max(dim=1)[1]
            preds_perturbed = perturbed_logits.max(dim=1)[1]
            count += batch_size * 2
            train_true.append(label.cpu().numpy())
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
            train_pred.append(preds_perturbed.detach().cpu().numpy())
            
        if args.scheduler == "cos":
            scheduler.step()
        elif args.scheduler == "step":
            if opt.param_groups[0]["lr"] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]["lr"] < 1e-5:
                for param_group in opt.param_groups:
                    param_group["lr"] = 1e-5
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        outstr = "Epsilon %.2f, Epoch %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f" % (
            e, epoch,
            train_loss * 1.0 / count,
            metrics.accuracy_score(train_true, train_pred),
            metrics.balanced_accuracy_score(train_true, train_pred),
        )
        print(outstr)
        accuracies[e] = metrics.accuracy_score(train_true, train_pred)
accuracies

Let's use 2 GPUs!
Epsilon 0.00, Epoch 0, loss: 1.554986, train acc: 0.571203, train avg acc: 0.395274
Epsilon 0.00, Epoch 1, loss: 0.883579, train acc: 0.729540, train avg acc: 0.602130
Epsilon 0.00, Epoch 2, loss: 0.720651, train acc: 0.775193, train avg acc: 0.670126
Epsilon 0.02, Epoch 0, loss: 1.714395, train acc: 0.529469, train avg acc: 0.345983
Epsilon 0.02, Epoch 1, loss: 1.023097, train acc: 0.692335, train avg acc: 0.556616
Epsilon 0.02, Epoch 2, loss: 0.871743, train acc: 0.733001, train avg acc: 0.614595
Epsilon 0.05, Epoch 0, loss: 1.752846, train acc: 0.515421, train avg acc: 0.336389
Epsilon 0.05, Epoch 1, loss: 1.089082, train acc: 0.676252, train avg acc: 0.535293
Epsilon 0.05, Epoch 2, loss: 0.915142, train acc: 0.718852, train avg acc: 0.592432
Epsilon 0.10, Epoch 0, loss: 1.809684, train acc: 0.506871, train avg acc: 0.327282
Epsilon 0.10, Epoch 1, loss: 1.129660, train acc: 0.659914, train avg acc: 0.518961
Epsilon 0.10, Epoch 2, loss: 0.960637, train acc: 0.706230

{0: 0.7751934039087948,
 0.02: 0.7330008143322475,
 0.05: 0.7188517915309446,
 0.1: 0.7062296416938111,
 0.2: 0.7074002442996743}

In [7]:
mod = DGCNN_cls(args).to(device)
mod = nn.DataParallel(mod)
mod.load_state_dict(torch.load(args.model_path, map_location=torch.device(device)))

train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
    )
print("getting sample")
sample = None
label = None
for data, l in train_loader:
    print(0)
    data, l = data.to(device), l.to(device).squeeze()
    sample, label = data, l
    break


getting sample
0


In [8]:
vis_point_cloud(sample[0].cpu())

In [9]:
sample[0].cpu().size()

torch.Size([1024, 3])